### Вариант 4

«Алгоритм» с селективным повторением

Под «алгоритмом» с селективным повторением подразумевается не конкретный алгоритм, а некоторая идея, которая может быть реализована различными способами. В рассмотренном ранее алгоритме с возвратом, источник в некоторых случаях повторно передавал сообщения, которые были приняты без ошибок. Что бы избавиться от данного недостатка, на приемной стороне можно использовать буфер для хранения верно принятых сообщений и повторно передавать только те сообщения, которые были приняты с ошибками, либо те, которые не поместились в буфер. Поэтому алгоритм с селективным повторением работает следующим образом. Источник непрерывно передает сообщения, не дожидаясь квитанции на отправленное ранее сообщение. При задержке получения квитанции равной τ единицам времени, после передачи сообщения, абонент  успевает передать еще τ сообщений, до того как получит квитанцию на него. При получении отрицательного квитанции на сообщение, источник повторяет передачу этого сообщения. Приемник после отправки отрицательной квитанции сохраняет принятые без ошибок последующие сообщения в буфер. При получении цепочки последовательно идущих сообщений, все они передаются на вышележащий уровень. Если в буфере нет места для принятого верно сообщения, то оно удаляется, а по обратному каналу передается отрицательная квитанция. Пример реализации этой идеи при τ = 2 и длине буфера равной 3 приведен на рис. 12.

При данном методе передачи возможны различные подходы работы с буфером.

Написать моделирующую программу для алгоритма с селективными повторениями, описанного в теоретических материалах. Размер буфера на приемнике L > 1. Стратегия работы с буфером: в буфер записываются только сообщения с номерами i+1, i+1, ..., i+L, при ожидании сообщения с номером i. Сообщения выдаются пользователю только в соответствии с правильным порядком следования, определяемого нумерацией на источнике.

In [2]:
import random
from dataclasses import dataclass
from typing import List, Optional, Dict
from collections import deque

@dataclass
class Message:
    id: int
    content: str
    is_error: bool = False
    
@dataclass
class Acknowledgment:
    message_id: int
    is_positive: bool

class Sender:
    def __init__(self, tau: int):
        self.tau = tau  # Задержка получения квитанции
        self.next_message_id = 1
        self.unacknowledged_messages: Dict[int, Message] = {}
        
    def send_message(self) -> Message:
        """Отправка нового сообщения"""
        message = Message(
            id=self.next_message_id,
            content=f"Message_{self.next_message_id}"
        )
        self.unacknowledged_messages[message.id] = message
        self.next_message_id += 1
        return message
    
    def resend_message(self, message_id: int) -> Optional[Message]:
        """Повторная отправка сообщения по его ID"""
        if message_id in self.unacknowledged_messages:
            return self.unacknowledged_messages[message_id]
        return None

class Receiver:
    def __init__(self, buffer_size: int):
        self.buffer_size = buffer_size
        self.buffer: Dict[int, Message] = {}  # Буфер для сообщений
        self.expected_message_id = 1  # ID следующего ожидаемого сообщения
        self.delivered_messages: List[Message] = []  # Доставленные сообщения
        
    def receive_message(self, message: Message) -> Acknowledgment:
        """Обработка входящего сообщения"""
        # Симулируем ошибку при передаче с вероятностью 0.2
        if random.random() < 0.2:
            message.is_error = True
            return Acknowledgment(message.id, is_positive=False)
            
        # Если сообщение пришло с ошибкой
        if message.is_error:
            return Acknowledgment(message.id, is_positive=False)
            
        # Если это ожидаемое сообщение
        if message.id == self.expected_message_id:
            self.delivered_messages.append(message)
            self.expected_message_id += 1
            
            # Проверяем буфер на наличие последовательных сообщений
            while self.expected_message_id in self.buffer:
                self.delivered_messages.append(self.buffer.pop(self.expected_message_id))
                self.expected_message_id += 1
                
            return Acknowledgment(message.id, is_positive=True)
            
        # Если это будущее сообщение и оно попадает в окно буфера
        if (message.id > self.expected_message_id and 
            message.id < self.expected_message_id + self.buffer_size):
            if len(self.buffer) < self.buffer_size:
                self.buffer[message.id] = message
                return Acknowledgment(message.id, is_positive=True)
                
        # Если буфер переполнен или сообщение вне окна
        return Acknowledgment(message.id, is_positive=False)

class SelectiveRepeatSimulation:
    def __init__(self, tau: int, buffer_size: int, simulation_time: int):
        self.tau = tau
        self.buffer_size = buffer_size
        self.simulation_time = simulation_time
        self.sender = Sender(tau)
        self.receiver = Receiver(buffer_size)
        self.acknowledgment_queue = deque()
        self.current_time = 0
        
    def run(self):
        """Запуск симуляции"""
        print(f"\nЗапуск симуляции с параметрами:")
        print(f"tau = {self.tau}")
        print(f"размер буфера = {self.buffer_size}")
        print(f"время симуляции = {self.simulation_time}\n")
        
        while self.current_time < self.simulation_time:
            print(f"\nВремя: {self.current_time}")
            
            # Отправка нового сообщения
            message = self.sender.send_message()
            print(f"Отправлено сообщение {message.id}")
            
            # Получение сообщения и отправка квитанции
            ack = self.receiver.receive_message(message)
            self.acknowledgment_queue.append((self.current_time + self.tau, ack))
            
            # Обработка пришедших квитанций
            while self.acknowledgment_queue and self.acknowledgment_queue[0][0] <= self.current_time:
                _, ack = self.acknowledgment_queue.popleft()
                if not ack.is_positive:
                    resent_message = self.sender.resend_message(ack.message_id)
                    if resent_message:
                        print(f"Повторная отправка сообщения {resent_message.id}")
                        new_ack = self.receiver.receive_message(resent_message)
                        self.acknowledgment_queue.append((self.current_time + self.tau, new_ack))
            
            # Вывод состояния системы
            print(f"Буфер приемника: {list(self.receiver.buffer.keys())}")
            print(f"Доставленные сообщения: {[msg.id for msg in self.receiver.delivered_messages]}")
            
            self.current_time += 1

simulation = SelectiveRepeatSimulation(
    tau=2,  # Задержка получения квитанции
    buffer_size=3,  # Размер буфера
    simulation_time=20  # Время симуляции
)
simulation.run()


Запуск симуляции с параметрами:
tau = 2
размер буфера = 3
время симуляции = 20


Время: 0
Отправлено сообщение 1
Буфер приемника: []
Доставленные сообщения: []

Время: 1
Отправлено сообщение 2
Буфер приемника: [2]
Доставленные сообщения: []

Время: 2
Отправлено сообщение 3
Повторная отправка сообщения 1
Буфер приемника: [2, 3]
Доставленные сообщения: []

Время: 3
Отправлено сообщение 4
Буфер приемника: [2, 3]
Доставленные сообщения: []

Время: 4
Отправлено сообщение 5
Повторная отправка сообщения 1
Буфер приемника: [2, 3]
Доставленные сообщения: []

Время: 5
Отправлено сообщение 6
Повторная отправка сообщения 4
Буфер приемника: [2, 3]
Доставленные сообщения: []

Время: 6
Отправлено сообщение 7
Повторная отправка сообщения 5
Повторная отправка сообщения 1
Буфер приемника: [2, 3]
Доставленные сообщения: []

Время: 7
Отправлено сообщение 8
Повторная отправка сообщения 6
Повторная отправка сообщения 4
Буфер приемника: [2, 3]
Доставленные сообщения: []

Время: 8
Отправлено сообщение 9
Повт